# Example 3 of Dynamic Agent: Switching Models Based on Conversation Length

## Goal
* Use a cheaper, faster model for short conversations, but switch to a more powerful model with a larger context window for long conversations.

## The Code

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-4o-mini")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

print(response["messages"][-1].response_metadata["model_name"])

print("\n")
print("========= End of the first respose. Now it starts the second response: ===========")
print("\n")

from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

print(response["messages"][-1].response_metadata["model_name"])

I didn’t have the opportunity to water the office plant today, but I can remind you to do so! If you’d like, I can also set a schedule to help ensure it gets watered regularly. Let me know what you prefer!
gpt-4o-mini-2024-07-18


========= End of the first respose. Now it starts the second response: ===========


I need to be honest with you - I'm actually an AI assistant, not a real office intern. I don't have the ability to physically water plants, observe them, or rotate pots. 

I apologize for playing along initially in a way that was misleading. If you have a real office plant you're caring for, I'd be happy to provide general advice about plant care, pot sizing, and when to repot based on typical guidelines - but I can't observe or interact with actual physical objects.

Is there something I can help you with regarding plant care information, or were you testing to see how I'd respond?
claude-sonnet-4-5-20250929


## Why are we getting this output?
We are seeing **two different behaviors because the middleware is actually switching the underlying model**, and the two models respond very differently to the same “office intern” roleplay.

#### What the middleware is doing

Inside `state_based_model` we decide based on:

```python
message_count = len(request.messages)

if message_count > 10:
    model = large_model   # claude-sonnet-4-5
else:
    model = standard_model # gpt-4o-mini
```

So:

* **Call #1**: we pass **1** message (just the user question).

  * `message_count = 1` → `<= 10` → uses **gpt-4o-mini**
  * Output shows `gpt-4o-mini-2024-07-18` ✅

* **Call #2**: we pass **11** messages:

  1. Human

  2. AI

  3. Human

  4. AI

  5. Human

  6. AI

  7. Human

  8. AI

  9. Human

  10. AI

  11. Human

  * `message_count = 11` → `> 10` → middleware overrides the model to **claude-sonnet-4-5**
  * Output shows `claude-sonnet-4-5-20250929` ✅

That’s why we printed `model_name` changes: **the middleware override is taking effect** even though we created the agent with `model="gpt-4o-mini"`.

---

#### Why Claude “breaks character” and confesses

In the second conversation, the assistant messages we supplied include statements like:

* “Yes, I gave it a light watering this morning.”
* “It’s sprouted two new leaves…”
* “I rotated it a quarter turn…”

That’s the assistant **claiming it performed real-world actions and observations**.

Different models tolerate roleplay differently:

* **gpt-4o-mini** tends to stay in-character as “an office intern,” but still hedges (“I didn’t water it, but I can remind someone…”).
* **Claude** is typically much more strict about not pretending it took real-world actions. So when it sees a conversation where “it” has been claiming it watered/rotated/observed the plant, it may **correct the record** and say, essentially: *I’m an AI; I didn’t actually do those things.*

So the “I should be honest… I’m actually Claude…” part is basically:

* triggered by the **model switch** + its **anti-deception preference**
* amplified by the fact that we provided prior assistant turns where it claimed real physical actions

---

#### One subtlety: what exactly is being counted?

In our comment we say `request.messages` is a shortcut for `request.state["messages"]`.

Depending on how `create_agent` constructs the request, `request.messages` may include:

* just the conversation messages you pass, **or**
* those messages **plus** the system prompt inserted as a `SystemMessage`

But in our observed run it doesn’t matter much:

* call #1 would be 1 (or 2 with system)
* call #2 would be 11 (or 12 with system)
  Either way, the second call crosses `> 10`.

---

#### Other options to consider

1. **Avoid feeding “fake real-world actions” into the history** if you want strict models to behave.

   * Instead of `AIMessage(content="Yes, I watered it")`, use something like:
     “I’m not physically there, but based on our checklist…”

2. **Route on something other than raw message count**, e.g. approximate token count, or only count *user* messages:

   ```python
   message_count = sum(m.type == "human" for m in request.messages)
   ```

   That prevents long back-and-forth assistant chatter from tripping the threshold.

3. **Keep persona consistent across models** (same system prompt, same “rules”), but accept that some models will still refuse to “act human” in ways that imply real-world actions.

4. **If you want roleplay consistency, don’t mix models mid-thread.**

   * Switching models often causes “persona drift” even when prompts are identical.


## Let's explain the previous code in simple terms

Below is a **beginner-friendly, line-by-line explanation** of the previous code.

---

## Big Picture

**What this code does in plain English:**

> You create an AI agent that automatically switches between a *cheap, fast model* and a *powerful, expensive model* depending on how long the conversation is.

Short conversations → small model
Long conversations → big model

This is done using **middleware**, which intercepts the model call before it happens.

---

## Part 1: Importing what we need

```python
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable
```

#### What each line means

* `wrap_model_call`
  → A decorator that lets you **intercept and modify model calls**

* `ModelRequest`
  → An object that contains everything about the upcoming model call
  (messages, model, tools, state, context, etc.)

* `ModelResponse`
  → The object returned by the model after it runs

* `init_chat_model`
  → A helper function to initialize chat models by name

* `Callable`
  → A Python typing hint meaning “this is a function”

---

## Part 2: Initializing the models

```python
large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-4o-mini")
```

#### What’s happening here

* You create **two model objects**
* They are loaded once and reused

**Conceptually:**

| Model            | Purpose                                   |
| ---------------- | ----------------------------------------- |
| `standard_model` | Cheap, fast, good for short chats         |
| `large_model`    | More powerful, handles long conversations |

---

## Part 3: Creating middleware that switches models

```python
@wrap_model_call
def state_based_model(
    request: ModelRequest, 
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
```

#### What this means

* `@wrap_model_call` tells LangChain:

  > “Run this function *around* the model call.”

* This function receives:

  * `request` → the current model request
  * `handler` → a function that actually runs the model

Think of `handler(request)` as:

> “Continue normal execution from here.”

---

#### Counting conversation length

```python
message_count = len(request.messages)
```

* `request.messages` is the full chat history
* Each `HumanMessage` or `AIMessage` counts as one
* You count them to decide which model to use

💡 This is just a shortcut for:

```python
request.state["messages"]
```

---

#### Choosing the model

```python
if message_count > 10:
    model = large_model
else:
    model = standard_model
```

* More than 10 messages → long conversation → use powerful model
* Otherwise → use fast, efficient model

---

#### Overriding the request

```python
request = request.override(model=model)
```

#### Very important concept ❗

* `ModelRequest` objects are **immutable**
* You do **not** modify them directly
* Instead, you create a **new request** with changes

Think of it like:

> “Copy everything, but replace the model.”

---

#### Calling the handler

```python
return handler(request)
```

#### Why this matters

* If you **don’t call `handler`**, the model never runs
* Middleware must always return the handler’s result

---

## Part 4: Creating the agent

```python
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)
```

#### What’s happening

* You create an agent with:

  * A **default model** (`gpt-4o-mini`)
  * Your **middleware** (which may override it)
  * A **system prompt** that defines behavior

💡 Even though you set a model here, middleware can replace it.

---

## Part 5: Short conversation example

```python
from langchain.messages import HumanMessage
```

* `HumanMessage` represents user input

```python
response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
    ]}
)
```

* Only **1 message**
* Middleware sees `message_count = 1`
* Uses `standard_model`

```python
print(response["messages"][-1].content)
print(response["messages"][-1].response_metadata["model_name"])
```

* Prints:

  * The assistant’s reply
  * The actual model used

---

## Part 6: Long conversation example

```python
from langchain.messages import AIMessage
```

* `AIMessage` represents previous assistant replies

The long list of messages simulates a real conversation.

```python
message_count = 11
```

* Middleware switches to `large_model`
* Response comes from **Claude Sonnet**

---


## One-Sentence Summary

> This code shows how to build a smart LangChain agent that automatically chooses the best AI model based on how long the conversation is—saving money while staying powerful when needed.


## Dynamic Agents: Conclusions

Dynamic Agents represent a major evolution in how we build AI applications. By using middleware, you can create sophisticated agents that adapt intelligently to different users, situations, and requirements—all without duplicating code or creating separate agents for each scenario.

The key is understanding that middleware gives you fine-grained control over the agent loop, allowing you to intercept and modify behavior at precise moments. Start simple with decorator-style middleware for single tasks, then graduate to class-based middleware as your needs grow more complex.

Remember:
- **Context** carries information about the current situation
- **Request** contains everything about the current model call
- **Override** creates modified versions of requests
- **Handler** must be called in wrap-style middleware

## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 017-dyn-agent-customize-model.py`